In [7]:
import pickle
import os 
import glob
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import plyfile
import numpy as np
import open3d as o3d
import h5py
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import open3d as o3d
from sklearn.decomposition import PCA
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances

import cv2
import numpy as np

import os.path as osp
import sys
ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import evaluation, scan3r


data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
curr_scan_id = "0958222a-e2c2-2de1-9474-35e601b3682a"#"0f2f271b-b736-2f71-8f3a-e9265463abcb" #"02b33dfb-be2b-2d54-92d2-cd012b2b3c40"#"02b33e01-be2b-2d54-93fb-4145a709cec5" 
new_scan_id =  "0958222a-e2c2-2de1-9474-35e601b3682a"#"0f2f271d-b736-2f71-8d77-dfc428e30c1e" #"fcf66d8a-622d-291c-8429-0e1109c6bb26" #"fcf66d8a-622d-291c-8429-0e1109c6bb26"


new_scan_id =   "63b87cef-ef3f-28f2-87a7-81c3fffbfa2d"
curr_scan_id = scan3r.get_reference_id(data_dir,new_scan_id)
print(curr_scan_id)



/local/home/ekoller/BT
63b87cf1-ef3f-28f2-871a-c1551f129ce6


In [2]:
#for a given scene get the colours of the differnt object_ids
def get_id_colours(data_dir,scan_id):
    #access the mesh file to get the colour of the ids
    mesh_file = osp.join(data_dir,"scenes", scan_id, "labels.instances.annotated.v2.ply")
    ply_data = plyfile.PlyData.read(mesh_file)
    # Extract vertex data
    vertices = ply_data['vertex']
    vertex_count = len(vertices)
    
    # Initialize dictionary to store object_id -> color mappings
    object_colors = {}
    
   # Iterate through vertices
    for i in range(vertex_count):
        vertex = vertices[i]
        object_id = vertex['objectId']
        color = (vertex['blue'], vertex['green'],vertex['red'])
        
        # Check if object_id already in dictionary, otherwise initialize a Counter
        if object_id in object_colors:
            object_colors[object_id][color] += 1
        else:
            object_colors[object_id] = Counter({color: 1})
    
    # Convert Counter to dictionary with most frequent color
    for object_id, color_counter in object_colors.items():
        most_common_color = color_counter.most_common(1)[0][0]
        object_colors[object_id] = np.array(most_common_color)
    
    return object_colors

# colors = get_id_colours(data_dir,curr_scan_id)
# print("cur scan id colors", colors)
# colors = get_id_colours(data_dir,new_scan_id)
# print("new scan id colors", colors.keys())

In [3]:
def acces_predicted_objects(scan_id):
    all_centers = {}
    filename = osp.join("/media/ekoller/T7/Predicted_Centers", scan_id + ".h5")
    with h5py.File(filename, 'r') as h5file:
        for obj_id in h5file.keys():
            obj_group = h5file[obj_id]
            center = np.array(obj_group['center'])
            points = np.array(obj_group['points'])
            votes = int(obj_group['votes'][()])
            
            # Add it back to the dictionary
            all_centers[int(obj_id)] = {
                'center': center,
                'points': points,
                'votes': votes
            }
    return all_centers

#acces_predicted_objects(new_scan_id)

this segment displays the predicted pointcloud and the ground truth point cloud

In [ ]:
# create the newly predicted pointcloud
all_centers = acces_predicted_objects( new_scan_id)
#initialize the final pointcloud

final_point_cloud = o3d.geometry.PointCloud()

#colour
obj_colors = get_id_colours(data_dir, new_scan_id)
print("colors dict", obj_colors.keys())
obj_colors_ref = get_id_colours(data_dir, curr_scan_id)

# Iterate over all objects in all_centers
for obj_id, data in all_centers.items():
   
        print("hey object added" , obj_id,  len(data["points"]))

        if obj_id not in obj_colors:
            color = np.array(obj_colors_ref[obj_id], dtype=np.float32)
            color = np.reshape(color, (3,))
            color = color / 255.0

        else: 
                
            points = data['points']  # Get points
            color = np.array(obj_colors[obj_id], dtype=np.float32)
            color = np.reshape(color, (3,))
            color = color / 255.0

        #color = np.array(obj_colors[obj_id])/255.0 # Generate a unique color for this object ID
        #color = np.random.rand(3)
        # Create a point cloud for the current object's points
        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(points)

        # Create a color array of the same size as the number of points, and assign the same color to all points
        #colors = np.tile(color, (points.shape[0], 1))  # Shape (num_points, 3)
        point_cloud.colors = o3d.utility.Vector3dVector(np.tile(color, (len(points), 1)))

        #colors = np.asarray(point_cloud.colors)

        # # Scale the colors by the brightness factor
        # adjusted_colors = np.clip(colors * 1.3, 0, 1)  # Clip to ensure values are in the [0, 1] range

        # Update the point cloud's colors
        # point_cloud.colors = o3d.utility.Vector3dVector(adjusted_colors)

        # Add the current object's point cloud to the final point cloud
        final_point_cloud += point_cloud

      
        center = data["center"]  # Get the center point

        # print(color)
        # Create a sphere mesh centered at `data["center"]`
        sphere_radius = 0.15  # Adjust the radius of the sphere
        sphere_mesh = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
        sphere_mesh.translate(center)  # Move the sphere to the center point

        # Sample points from the mesh surface to create a point cloud
        sphere_pcd = sphere_mesh.sample_points_poisson_disk(number_of_points=500)  # Adjust number of points if needed

        # Set a unique color for the sphere (e.g., red)

        sphere_pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (len(sphere_pcd.points), 1)))

        # Add the sphere point cloud to the final point cloud
        final_point_cloud += sphere_pcd



#also access the ground truth pointclouds
pklfile = osp.join('/local/home/ekoller/R3Scan', 'files', 'orig', 'data', '{}.pkl'.format(new_scan_id))

with open(pklfile, "rb") as f:
    # Load the data from the pickle file
    data = pickle.load(f)
    
    # Extract object points and IDs from the pickle data
    pointclouds = data['obj_points'][512]
    object_ids = data['objects_id']
    
    # Create a new point cloud object for the .pkl data
    pkl_point_cloud = o3d.geometry.PointCloud()

    # Iterate over all the points and their object IDs
    # for obj_id, points in zip(object_ids, pointclouds):
    #         # Assign a default color (e.g., black) to these objects
    #         #print("object id", obj_id)
    #         color = obj_colors[obj_id]/225.0
    #         #print("clour", obj_colors[obj_id]/225.0)
    #         # Create a point cloud for this object's points
    #         pcd = o3d.geometry.PointCloud()
    #         pcd.points = o3d.utility.Vector3dVector(points)
    #         pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (points.shape[0], 1)))  # Assign the color

    #         # Add this point cloud to the combined pkl_point_cloud
    #         pkl_point_cloud += pcd

colors = np.asarray(pkl_point_cloud.colors)

# Scale the colors by the brightness factor
#adjusted_colors = np.clip(colors *0.7, 0, 1)  # Clip to ensure values are in the [0, 1] range

# Update the point cloud's colors
# pkl_point_cloud.colors = o3d.utility.Vector3dVector(adjusted_colors)
pkl_point_cloud.colors = o3d.utility.Vector3dVector(colors)
# Now combine both point clouds (final_point_cloud and pkl_point_cloud) for visualization
combined_point_cloud = final_point_cloud + pkl_point_cloud


new_scan_id_d =  curr_scan_id

#also display the mesh such that it is easier to recognize what is new
pathToMesh = osp.join(data_dir,"scenes", new_scan_id_d, "labels.instances.align.annotated.v2.ply")
new_mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#check if it has colours
if not new_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(new_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(new_mesh.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
new_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)



new_scan_id_d =  new_scan_id

#also display the mesh such that it is easier to recognize what is new
pathToMesh = osp.join(data_dir,"scenes", new_scan_id_d, "labels.instances.align.annotated.v2.ply")
new_mesh_f = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#check if it has colours
if not new_mesh_f.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(new_mesh_f.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(new_mesh_f.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
new_mesh_f.vertex_colors = o3d.utility.Vector3dVector(colors)

# Visualize both together using Open3D
o3d.visualization.draw_geometries([final_point_cloud], window_name="Combined Point Clouds from Scan and PKL")



#o3d.visualization.draw_geometries([final_point_cloud], window_name="Combined Point Cloud")



colors dict dict_keys([15, 6, 4, 3, 2, 14, 5, 13, 11, 1, 0, 7, 16, 8, 12, 17, 10])
hey object added 1 1318
hey object added 10 863
hey object added 11 585
hey object added 12 897
hey object added 13 829
hey object added 14 576
hey object added 15 633
hey object added 16 421
hey object added 17 94
hey object added 2 689
hey object added 3 103
hey object added 5 1797
hey object added 6 604
hey object added 7 2026


In [116]:
#get scans which have differences and moved objects

def is_in_boundingbox( center, boundingbox):
        min_coords = np.min(boundingbox, axis=0)
        max_coords = np.max(boundingbox, axis=0)

     
        is_inside = (np.all(min_coords <= center) and np.all(center <= max_coords))

        # if is_inside:
        #     print("the object was inside")
        return is_inside


n_scan_id = "ddc737a9-765b-241a-9dd7-bf14079f4804"
c_scan_id = scan3r.get_reference_id(data_dir, n_scan_id)

pklfile = osp.join('/local/home/ekoller/R3Scan', 'files', 'orig', 'data', '{}.pkl'.format(c_scan_id))

with open(pklfile, "rb") as f:
    # Load the data from the pickle file
    data = pickle.load(f)
    
    # Extract object points and IDs from the pickle data
    c_boundingboxes = data["bounding_boxes"]
    

pklfile = osp.join('/local/home/ekoller/R3Scan', 'files', 'orig', 'data', '{}.pkl'.format(n_scan_id))

with open(pklfile, "rb") as f:
    # Load the data from the pickle file
    data = pickle.load(f)
    
    # Extract object points and IDs from the pickle data
    n_obj_centers = data['object_centers']
    n_object_ids = data['objects_id']

    for obj_id, points in zip(n_object_ids, n_obj_centers):
         if obj_id in c_boundingboxes.keys():
            if not is_in_boundingbox(n_obj_centers[obj_id], c_boundingboxes[obj_id]):
                print("obj id has mooved", obj_id)








    

obj id has mooved 9
